# 哈希

**dict** & **set** 依赖 **hash function**，达到 O(1) lookup time

为类 实现 `__hash__()`、`__eq__()`

注意事项
1. 保证 `__hash__()` 魔术方法的结果 不变 (对于自定义类，即使会改变，Python解释器不会报错)，数据初始化了就应该 immutable，如果要改应该生成一个新对象
2. dict、list、set 都是 mutable 因此 unhashable(不能调用 `__hash__()`)
3. str、byte、frozenset、tuple(元素也必须 immutable) 都是 immutable 因此 hashable(可以调用 `__hash__()`)
4. dict 的 key 必须 hashable， value 不需要
5. `__hash__()` 只会 在对象被插入 dict、set 时调用一次

类的可变性并不直接阻止你为类实现 `__hash__()` 方法，意味着即使一个类是 mutable，你也可以定义一个``__hash__()` 方法来使其实例 hashable

然而，如果类的实例在作为 字典键 或 集合元素 时被修改，它可能会导致数据结构的内部状态出现问题，丢失数据 或 无法找到键

通过重写 `__setattr__()` 方法，保证 immutable

或者使用 [attrs库](https://github.com/python-attrs/attrs) / [immutables库](https://github.com/MagicStack/immutables)




**规则**
1. if a==b, then hash(a)==hash(b)
2. if hash(a)==hash(b), then a might == b (可能 哈希冲突)
3. if hash(a)!=hash(b), then a!=b (规则1 的逆否命题)


Python3 所有类 都继承 Object类

Object类有 `__hash__()` & `__eq__()` 方法

一旦`__eq__()`被重写，Python需要确保`__hash__()`也被适当地定义以保持哈希表的一致性

重写`__eq__()`而不重写`__hash__()`时，Python会将你的类的`__hash__()`方法设置为**None**，意味着你的类的实例将不再是可哈希的，防止违反上述的哈希一致性原则

如果你的类的实例需要用作哈希表的键（例如字典的键或集合的元素），你需要**同时重写**`__eq__()`和`__hash__()`方法



哈希 != 加密

哈希不可逆，加密可逆

![](Pics/algo001.png)

安全的哈希算法
1. 碰撞概率低
2. 不能通过哈希值反推数据

常用 哈希算法
1. MD4
2. MD5       - 128 bits
3. SHA-1     - 160 bits
4. SHA-256   - 256 bits
5. SHA-512   - 512 bits

## 哈希表

哈希冲突时，用链表往后链接

![](Pics/algo002.png)

哈希表数组太小将会降级为纤细搜索(链表便利)

## Python 中的哈希 

In [12]:
a = "abcd"
print(hash(a))
print(hash("abcd"))
print(str.__hash__(a))
print(str.__hash__("abcd"))


8560444216108399403
8560444216108399403
8560444216108399403
8560444216108399403


In [13]:
# print(hash([1,2,3]))
# TypeError: unhashable type: 'list'


In [14]:
from typing import Any


class Person:
    def __init__(self, name, age):
        # 无法绕过 __setattr__ 的 raise
        # self.name = name
        # self.age = age
        # 可以绕过 __setattr__ 的 raise
        object.__setattr__(self, 'name', name)
        object.__setattr__(self, 'age', age)

    def __hash__(self):  # 保证 hashable，在 dict 中会使用
        hash_num = hash((self.name, self.age))
        print(hash_num)
        return hash_num

    def __eq__(self, other):
        if isinstance(other, Person):  # 先判断是否为相同数据类型
            return self.name == other.name and self.age == other.age
        return False

    def __setattr__(self, name: str, value: Any) -> None:
        raise TypeError("Class - <"+ self.__class__.__name__ + "> is immutable")

alice1 = Person("Alice", 30)
bob = Person("Bob", 30)
alice2 = Person("Alice", 30)

print(alice1 == alice2)  # True 调用 __eq__()魔术方法
print(alice1 is alice2)  # False is查看内存地址，不同实例，地址不同
print(alice1 == bob)     # False


person_dict = {}
person_dict[alice1] = "Data for Alice"

print(person_dict[alice2])

# alice1.age = 32  # TypeError: Class - <Person> is immutable


True
False
False
-8895997092314717297
-8895997092314717297
Data for Alice
